<a href="https://colab.research.google.com/github/alierenc/di725-transformers-and-attention-based-deep-networks-term-project/blob/main/Phase%20III%20/1.1.%20PaliGemma%20-%20Zero-Shot%20Image%20Captioning%20Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import login
hf_token = " " # Huggingface token
login(token = hf_token)

In [ ]:
!pip install -U datasets

In [ ]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from datasets import load_dataset, DatasetDict
import torch

# Load the dataset of full riscm
ds = load_dataset('caglarmert/full_riscm')

full = ds["train"]

# test   = indices [0, 3150)
test_ds = full.select(range(3150))

# validation = indices [3150, 6300)
val_ds = full.select(range(3150, 6300))

# train  = indices [6300, end)
train_ds = full.select(range(6300, len(full)))

# bundle into a DatasetDict
ds = DatasetDict({
    "val": val_ds,
    "test": test_ds,
    "train": train_ds,
})

# Load the model and the processor
model_id = "google/paligemma-3b-mix-224"
processor = AutoProcessor.from_pretrained(model_id)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PaliGemmaForConditionalGeneration(
  (model): PaliGemmaModel(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(256, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
              )
              (layer_norm2): LayerNorm((1152,), eps=1e-06, elem

In [ ]:
from tqdm import tqdm

model.eval()

# We add <image> token upon the warning.
prompt = "<image> caption en"
predictions = []

for i in tqdm(range(len(ds["test"])), desc="Generating captions"):
    # Get the image
    image = ds["test"][i]["image"]

    # Preprocess image and prompt
    inputs = processor(image, prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move to GPU

    # Generate caption
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=30)

    # Decode output
    caption = processor.decode(output[0], skip_special_tokens=True)
    if caption.startswith(prompt):
        caption = caption[len(prompt):].strip()

    predictions.append(caption)

Generating captions: 100%|██████████| 3150/3150 [30:45<00:00,  1.71it/s]


In [ ]:
# Get the references
# Define a varible to store the reference captions
all_references = []
for i in tqdm(range(len(ds["test"])), desc="Collecting reference captions"):
    # Get the reference
    reference_per_sample = []
    for j in range(1,6):
        reference = ds["test"][i][f"caption_{j}"]
        reference_per_sample.append(reference)
        print(f"The reference caption_{j}:")
        print(repr(reference))

    print()
    all_references.append(reference_per_sample)

The reference caption_1:
'A gray plane on the runway and the lawn beside .'
The reference caption_2:
'A grey plane is on the runway by the lawn .'
The reference caption_3:
'There is an airplane on the runway with a large lawn by the runway .'
The reference caption_4:
'A plane is parked on the runway next to the grass .'
The reference caption_5:
'There is a plane on the runway beside the grass .'

The reference caption_1:
'Three small planes parked in a line on the airport and a big plane behind them .'
The reference caption_2:
'There are four aircraft on the open ground, The largest of which is three times as large as the smallest one .'
The reference caption_3:
'There are many planes of different sizes in a clearing .'
The reference caption_4:
'Four planes are parked on the runway .'
The reference caption_5:
'Four planes of different sizes were on the marked ground .'

The reference caption_1:
'A plane parked in a line on the airport with some marks .'
The reference caption_2:
'A whit

The reference caption_3:
'There are many planes of different sizes in a clearing .'
The reference caption_4:
'Four planes are parked on the open space .'
The reference caption_5:
'Four planes are in the clearing next to the grass .'

The reference caption_1:
'A plane parked on the open area and a runway behind it .'
The reference caption_2:
'There is a plane and several cars on the open ground .'
The reference caption_3:
'There is a plane in a clearing and there is a runway next to the plane .'
The reference caption_4:
'A plane is parked on the runway next to the grass .'
The reference caption_5:
'There is an airplane in the parking lot .'

The reference caption_1:
'Three planes parked in a line on the airport while some facilities in front of them .'
The reference caption_2:
'Three aircraft parked on the yellow indicating line, And there were rows of neat containers in front of the aircraft .'
The reference caption_3:
'There are many planes of the same size in a clearing, With many co

The reference caption_3:
'A complex airport with many runways and buildings built on a huge lawn surrounded by a residential area .'
The reference caption_4:
'An airport is on the open place surrounded by many buildings .'
The reference caption_5:
'Two roads intersect at the airport .'

The reference caption_1:
'An airport with some staggered runways and a terminal on the lawn and some buildings beside .'
The reference caption_2:
'This airport has only one runway .'
The reference caption_3:
'Several runways and buildings are scattered in the airport while every plane is on their own position .'
The reference caption_4:
'There are several planes on the tarmac with several runways in the airport .'
The reference caption_5:
'The airport has many roads of different lengths .'

The reference caption_1:
'An airport with two parallel runways, A white terminal and an open area on the lawn and some villages beside .'
The reference caption_2:
'There are many buildings near the airport, And no ve

The reference caption_5:
'Several buildings are between four baseball diamonds .'

The reference caption_1:
'Four baseball diamonds on the lawn with some trees and some buildings beside .'
The reference caption_2:
'There are five baseball fields in this picture .'
The reference caption_3:
'There are four baseball diamonds in the grass along the roads with many trees .'
The reference caption_4:
'There are four baseball diamonds on both side of a road .'
The reference caption_5:
'Several buildings are between four baseball diamonds .'

The reference caption_1:
'A baseball diamond on the lawn with some trees while a parking lot and some buildings beside .'
The reference caption_2:
'There is a railway near the baseball field .'
The reference caption_3:
'There is a simple baseball diamond in the grass along the roads surrounded by parking lots, Houses and many trees .'
The reference caption_4:
'There is a baseball diamond between many buildings and the trees .'
The reference caption_5:
'Man

The reference caption_3:
'The basketball court is between two lawns,With vegetation nearby .'
The reference caption_4:
'There is a basketball court on the open place between the grass .'
The reference caption_5:
'The grass is beside the basketball court .'

The reference caption_1:
'Two basketball courts next to a lake and a building beside .'
The reference caption_2:
'There is a basketball court above .'
The reference caption_3:
'There are two basketball courts near the buildings and the swimming pool .'
The reference caption_4:
'There are two basketball courts in the middle of grass next to some buildings with a road pass by .'
The reference caption_5:
'Two basketball courts are surrounded by grass .'

The reference caption_1:
'Six basketball courts on the lawn and a playground beside .'
The reference caption_2:
'There are two basketball courts of different sizes on the green land .'
The reference caption_3:
'The neatly arrayed  basketball courts on the lawn are next to the runway .'

...

The reference caption_3:
'The tennis court is on the grass next to some buildings and trees .'
The reference caption_4:
'Many buildings of different shapes are beside the tennis courts .'
The reference caption_5:
'There are some green trees and black buildings beside the tennis court .'

The reference caption_1:
'There is a tennis court in the playground .'
The reference caption_2:
'Tennis court on the lawn next to the building .'
The reference caption_3:
'The tennis court is on the grass next to some buildings and trees .'
The reference caption_4:
'The tennis courts are surrounded by grass .'
The reference caption_5:
'The tennis court is on a green meadow .'

The reference caption_1:
'The tennis court was shadowed by streetlights .'
The reference caption_2:
'Tennis court and swimming pool in the middle of a residential area surrounded by trees .'
The reference caption_3:
'The tennis court is on the grass next to some buildings and trees .'
The reference caption_4:
'Many buildings are 

The reference caption_3:
'The bare and green terrace is next to some trees .'
The reference caption_4:
'There are many plants on the terrace .'
The reference caption_5:
'There are some green trees beside the terraces .'

The reference caption_1:
'There are two houses with orange roofs by the terraces .'
The reference caption_2:
'A large patch of brown and green mixed terraces with a road in the middle .'
The reference caption_3:
'The bare and green terrace is next to some trees .'
The reference caption_4:
'Many green plants are on the terrace .'
The reference caption_5:
'There are some buildings and green trees beside the terraces .'

The reference caption_1:
'There is a winding path through the green terraces .'
The reference caption_2:
'A large layer of brown and green mixed terraces with several winding narrow paths .'
The reference caption_3:
'The bare terrace is next to some trees .'
The reference caption_4:
'There are many green plants on the terrace .'
The reference caption_5:
'

The reference caption_3:
'The bare and green terrace is next to some trees .'
The reference caption_4:
'The green plants are on the terrace .'
The reference caption_5:
'There are green meadows beside the terraces .'

The reference caption_1:
'There is a terrace filled with water .'
The reference caption_2:
'A large layer of brown and green mixed terraces with several winding narrow paths .'
The reference caption_3:
'There are bare and green terrace .'
The reference caption_4:
'There are many green plants on the terrace .'
The reference caption_5:
'There are many yellow terraces and green ones .'

The reference caption_1:
'There is a row of grey houses beside the terraces .'
The reference caption_2:
'A large layer of brown and green mixed terraces .'
The reference caption_3:
'There are bare and green terrace .'
The reference caption_4:
'There are many plants on the terrace .'
The reference caption_5:
'There are some yellow terraces and green ones .'

The reference caption_1:
'There is a

The reference caption_1:
'The thermal power station in this industrial park is emitting smoke .'
The reference caption_2:
'In the thermal power station, there are neatly planned plants, Equipment, and good greening  There are two cooling towers, one with steam and a chimney .'
The reference caption_3:
'The thermal power station is on the bare land next to some buildings .'
The reference caption_4:
'There are many buildings next to the thermal power station .'
The reference caption_5:
'There are many buildings on the thermal power station .'

The reference caption_1:
'In the upper left corner of the picture, there is a smoking thermal power station .'
The reference caption_2:
'There are workshops and machinery in the thermal power station  There are two cooling towers, one steaming .'
The reference caption_3:
'The thermal power station is on the grass next to some buildings with white mist above .'
The reference caption_4:
'There are many buildings of different shapes next to the therma

The reference caption_1:
'There are rows of tall trees in this wetland .'
The reference caption_2:
'Brown trivial bare land in the river in a wetland .'
The reference caption_3:
'The wetland consists of bare land, Green plants and water .'
The reference caption_4:
'There are many different areas of wetlands .'
The reference caption_5:
'There are many water areas of different sizes on the wetland .'

The reference caption_1:
'This wetland is light green overall with some green plants in the middle .'
The reference caption_2:
'There are patches of brown bare land in a lake in a wetland .'
The reference caption_3:
'The wetland consists of bare land, Green plants and water .'
The reference caption_4:
'There are many wetlands of different shapes and sizes .'
The reference caption_5:
'There are many water areas of different sizes on the wetland .'

The reference caption_1:
'There is a straight path through this wetland .'
The reference caption_2:
'There is a large area of \u200b\u200bdense b

In [ ]:
# Check the format of the reference captions
print(all_references[:5])

[['A gray plane on the runway and the lawn beside .', 'A grey plane is on the runway by the lawn .', 'There is an airplane on the runway with a large lawn by the runway .', 'A plane is parked on the runway next to the grass .', 'There is a plane on the runway beside the grass .'], ['Three small planes parked in a line on the airport and a big plane behind them .', 'There are four aircraft on the open ground, The largest of which is three times as large as the smallest one .', 'There are many planes of different sizes in a clearing .', 'Four planes are parked on the runway .', 'Four planes of different sizes were on the marked ground .'], ['A plane parked in a line on the airport with some marks .', 'A white plane was parked on the instruction line .', 'An airplane parked in an open area with many containers next to it .', 'A plane is parked on the open space .', 'There is 1 plane on the ground marked .'], ['A small plane and a big plane parked next to boarding bridges .', 'A white plan

In [ ]:
# Check the format of the predicted captions. Each sample starts with a new line
print(predictions[:5])

["A large jetliner sits proudly on the runway, its powerful engines roaring. The plane's wing extends gracefully, while the tail gracefully curves upward.", 'A group of four airplanes are parked on a runway, their wings resting on the ground. The runway is made of concrete and has white lines painted on', "A large jetliner sits proudly on the tarmac, its powerful engines idling. The plane's tail and wing extend gracefully, while the white lines on", 'An aerial view of an airport with several planes parked at gates. The tarmac is gray, and the runway is also gray. There are several jet bridges', 'Two airplanes are parked on the tarmac at an airport. The planes are white, with red and white stripes on their tails. The wing of the plane']


In [ ]:
# Try to clean the new lines from the predicted captions for a better evaluation at later stages
for i in range(len(predictions)):
    predictions[i] = predictions[i].lstrip(" caption en\n")

predictions[:5]

["A large jetliner sits proudly on the runway, its powerful engines roaring. The plane's wing extends gracefully, while the tail gracefully curves upward.",
 'A group of four airplanes are parked on a runway, their wings resting on the ground. The runway is made of concrete and has white lines painted on',
 "A large jetliner sits proudly on the tarmac, its powerful engines idling. The plane's tail and wing extend gracefully, while the white lines on",
 'An aerial view of an airport with several planes parked at gates. The tarmac is gray, and the runway is also gray. There are several jet bridges',
 'Two airplanes are parked on the tarmac at an airport. The planes are white, with red and white stripes on their tails. The wing of the plane']

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction

nltk.download('punkt_tab')
nltk.download('punkt')

# Tokenize references and predictions:
tokenized_refs = [
    [nltk.word_tokenize(ref.lower()) for ref in refs]
    for refs in all_references
]

tokenized_hyps = [nltk.word_tokenize(pred.lower()) for pred in predictions]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tokenized_refs[0]

[['a',
  'gray',
  'plane',
  'on',
  'the',
  'runway',
  'and',
  'the',
  'lawn',
  'beside',
  '.'],
 ['a', 'grey', 'plane', 'is', 'on', 'the', 'runway', 'by', 'the', 'lawn', '.'],
 ['there',
  'is',
  'an',
  'airplane',
  'on',
  'the',
  'runway',
  'with',
  'a',
  'large',
  'lawn',
  'by',
  'the',
  'runway',
  '.'],
 ['a',
  'plane',
  'is',
  'parked',
  'on',
  'the',
  'runway',
  'next',
  'to',
  'the',
  'grass',
  '.'],
 ['there',
  'is',
  'a',
  'plane',
  'on',
  'the',
  'runway',
  'beside',
  'the',
  'grass',
  '.']]

In [ ]:
# Sentence-level BLEU-2
smooth = SmoothingFunction().method1
for i, (refs_per_sample, hyp_tok) in enumerate(zip(tokenized_refs, tokenized_hyps)):
    scores = []
    for refs_tok in refs_per_sample:
        score = sentence_bleu(
            [refs_tok],
            hyp_tok,
            weights=(1/2, 1/2),
            smoothing_function=smooth
        )
        scores.append(score)
    max_score = max(scores)
    print(f"Example {i+1:2d} BLEU-2: {max_score*100:.2f}")

Example  1 BLEU-2: 16.67
Example  2 BLEU-2: 17.94
Example  3 BLEU-2: 8.91
Example  4 BLEU-2: 9.59
Example  5 BLEU-2: 11.74
Example  6 BLEU-2: 9.59
Example  7 BLEU-2: 17.62
Example  8 BLEU-2: 9.62
Example  9 BLEU-2: 15.54
Example 10 BLEU-2: 9.62
Example 11 BLEU-2: 6.78
Example 12 BLEU-2: 28.77
Example 13 BLEU-2: 17.94
Example 14 BLEU-2: 15.54
Example 15 BLEU-2: 19.18
Example 16 BLEU-2: 13.56
Example 17 BLEU-2: 15.90
Example 18 BLEU-2: 39.39
Example 19 BLEU-2: 22.74
Example 20 BLEU-2: 20.34
Example 21 BLEU-2: 20.34
Example 22 BLEU-2: 12.69
Example 23 BLEU-2: 8.97
Example 24 BLEU-2: 17.41
Example 25 BLEU-2: 21.44
Example 26 BLEU-2: 23.97
Example 27 BLEU-2: 16.61
Example 28 BLEU-2: 9.28
Example 29 BLEU-2: 9.62
Example 30 BLEU-2: 2.57
Example 31 BLEU-2: 13.61
Example 32 BLEU-2: 19.18
Example 33 BLEU-2: 19.03
Example 34 BLEU-2: 16.61
Example 35 BLEU-2: 16.61
Example 36 BLEU-2: 9.62
Example 37 BLEU-2: 14.38
Example 38 BLEU-2: 12.69
Example 39 BLEU-2: 23.97
Example 40 BLEU-2: 17.94
Example 41 

In [ ]:
# Corpus-level BLEU-2
# corpus_bleu expects list-of-list-of-tokens refs, and list-of-tokens hyps
corpus_score = corpus_bleu(
    tokenized_refs,
    tokenized_hyps,
    weights=(1/2, 1/2),
    smoothing_function=smooth
)
print(f"\nCorpus BLEU-2: {corpus_score*100:.2f}")


Corpus BLEU-2: 18.37


In [ ]:
# Sentence-level BLEU-3
smooth = SmoothingFunction().method1
for i, (refs_per_sample, hyp_tok) in enumerate(zip(tokenized_refs, tokenized_hyps)):
    scores = []
    for refs_tok in refs_per_sample:
        score = sentence_bleu(
            [refs_tok],
            hyp_tok,
            weights=(1/3, 1/3, 1/3),
            smoothing_function=smooth
        )
        scores.append(score)
    max_score = max(scores)
    print(f"Example {i+1:2d} BLEU-3: {max_score*100:.2f}")

Example  1 BLEU-3: 10.22
Example  2 BLEU-3: 10.48
Example  3 BLEU-3: 3.13
Example  4 BLEU-3: 3.20
Example  5 BLEU-3: 3.67
Example  6 BLEU-3: 3.20
Example  7 BLEU-3: 13.04
Example  8 BLEU-3: 3.29
Example  9 BLEU-3: 9.52
Example 10 BLEU-3: 3.29
Example 11 BLEU-3: 2.54
Example 12 BLEU-3: 18.08
Example 13 BLEU-3: 13.20
Example 14 BLEU-3: 9.52
Example 15 BLEU-3: 13.80
Example 16 BLEU-3: 4.03
Example 17 BLEU-3: 4.49
Example 18 BLEU-3: 25.52
Example 19 BLEU-3: 17.70
Example 20 BLEU-3: 5.29
Example 21 BLEU-3: 14.35
Example 22 BLEU-3: 9.95
Example 23 BLEU-3: 3.06
Example 24 BLEU-3: 6.72
Example 25 BLEU-3: 11.80
Example 26 BLEU-3: 16.01
Example 27 BLEU-3: 9.95
Example 28 BLEU-3: 3.17
Example 29 BLEU-3: 3.29
Example 30 BLEU-3: 1.37
Example 31 BLEU-3: 8.93
Example 32 BLEU-3: 13.80
Example 33 BLEU-3: 9.95
Example 34 BLEU-3: 9.95
Example 35 BLEU-3: 9.95
Example 36 BLEU-3: 3.29
Example 37 BLEU-3: 4.20
Example 38 BLEU-3: 3.86
Example 39 BLEU-3: 16.01
Example 40 BLEU-3: 4.86
Example 41 BLEU-3: 19.48
Ex

In [ ]:
# Corpus-level BLEU-3
# corpus_bleu expects list-of-list-of-tokens refs, and list-of-tokens hyps
corpus_score = corpus_bleu(
    tokenized_refs,
    tokenized_hyps,
    weights=(1/3, 1/3, 1/3),
    smoothing_function=smooth
)
print(f"\nCorpus BLEU-3: {corpus_score*100:.2f}")


Corpus BLEU-3: 8.91


In [ ]:
# Sentence-level BLEU-4
smooth = SmoothingFunction().method1
for i, (refs_per_sample, hyp_tok) in enumerate(zip(tokenized_refs, tokenized_hyps)):
    scores = []
    for refs_tok in refs_per_sample:
        score = sentence_bleu(
            [refs_tok],
            hyp_tok,
            weights=(1/4, 1/4, 1/4, 1/4),
            smoothing_function=smooth
        )
        scores.append(score)
    max_score = max(scores)
    print(f"Example {i+1:2d} BLEU-4: {max_score*100:.2f}")

Example  1 BLEU-4: 4.55
Example  2 BLEU-4: 4.54
Example  3 BLEU-4: 1.87
Example  4 BLEU-4: 1.87
Example  5 BLEU-4: 2.07
Example  6 BLEU-4: 1.87
Example  7 BLEU-4: 9.52
Example  8 BLEU-4: 1.94
Example  9 BLEU-4: 4.23
Example 10 BLEU-4: 1.94
Example 11 BLEU-4: 1.57
Example 12 BLEU-4: 6.84
Example 13 BLEU-4: 9.61
Example 14 BLEU-4: 4.23
Example 15 BLEU-4: 9.93
Example 16 BLEU-4: 2.22
Example 17 BLEU-4: 2.40
Example 18 BLEU-4: 15.75
Example 19 BLEU-4: 14.23
Example 20 BLEU-4: 2.72
Example 21 BLEU-4: 10.23
Example 22 BLEU-4: 4.37
Example 23 BLEU-4: 1.81
Example 24 BLEU-4: 4.28
Example 25 BLEU-4: 4.97
Example 26 BLEU-4: 11.10
Example 27 BLEU-4: 4.37
Example 28 BLEU-4: 1.87
Example 29 BLEU-4: 1.94
Example 30 BLEU-4: 1.00
Example 31 BLEU-4: 4.11
Example 32 BLEU-4: 9.93
Example 33 BLEU-4: 4.37
Example 34 BLEU-4: 4.37
Example 35 BLEU-4: 4.37
Example 36 BLEU-4: 1.94
Example 37 BLEU-4: 2.29
Example 38 BLEU-4: 2.15
Example 39 BLEU-4: 11.10
Example 40 BLEU-4: 2.55
Example 41 BLEU-4: 15.30
Example 42

In [ ]:
# Corpus-level BLEU-4
# corpus_bleu expects list-of-list-of-tokens refs, and list-of-tokens hyps
corpus_score = corpus_bleu(
    tokenized_refs,
    tokenized_hyps,
    weights=(1/4, 1/4, 1/4, 1/4),
    smoothing_function=smooth
)
print(f"\nCorpus BLEU-4: {corpus_score*100:.2f}")


Corpus BLEU-4: 4.12


In [ ]:
# Go on to calculate ROUGE scores
!pip install rouge-score

In [ ]:
import nltk
from collections import Counter

# Ensure tokenizer
nltk.download('punkt', quiet=True)

def rouge_n(ref: str, hyp: str, n: int = 4):
    ref_toks = nltk.word_tokenize(ref.lower())
    hyp_toks = nltk.word_tokenize(hyp.lower())
    ref_ngrams = list(nltk.ngrams(ref_toks, n))
    hyp_ngrams = list(nltk.ngrams(hyp_toks, n))
    ref_counts = Counter(ref_ngrams)
    hyp_counts = Counter(hyp_ngrams)
    overlap = sum(min(ref_counts[ng], hyp_counts[ng]) for ng in ref_counts)
    recall = overlap / max(len(ref_ngrams), 1)
    precision = overlap / max(len(hyp_ngrams), 1)
    f1 = 2 * recall * precision / (recall + precision + 1e-8)
    return (recall, precision, f1)


# Compute ROUGE-2
all_recalls, all_precisions, all_f1s = [], [], []
for refs, pred in zip(all_references, predictions):
    recalls_per_sample, precisions_per_sample, f1s_per_sample = [], [], []
    for ref in refs:
        r, p, f = rouge_n(ref, pred, n=2)
        recalls_per_sample.append(r)
        precisions_per_sample.append(p)
        f1s_per_sample.append(f)

    max_score = max(f1s_per_sample)
    max_index = f1s_per_sample.index(max_score)
    all_recalls.append(recalls_per_sample[max_index])
    all_precisions.append(precisions_per_sample[max_index])
    all_f1s.append(f1s_per_sample[max_index])
    print(f"REF:  {refs[max_index]!r}")
    print(f"HYP:  {pred!r}")
    print(f"   ROUGE-2 Recall:    {recalls_per_sample[max_index]*100:.2f}")
    print(f"   ROUGE-2 Precision: {precisions_per_sample[max_index]*100:.2f}")
    print(f"   ROUGE-2 F1:        {f1s_per_sample[max_index]*100:.2f}\n")

Streaming output truncated to the last 5000 lines.
   ROUGE-2 F1:        22.22

REF:  'There are several vehicles on the road at the roundabout, And many cars are parked on both sides of the road next to the roundabout .'
HYP:  'A roundabout with a large green circle in the middle. The roundabout has a white line painted on the road, a white bus on the road, and'
   ROUGE-2 Recall:    19.23
   ROUGE-2 Precision: 17.24
   ROUGE-2 F1:        18.18

REF:  'The roundabout with three exits and entrances is in the residential area .'
HYP:  ''
   ROUGE-2 Recall:    0.00
   ROUGE-2 Precision: 0.00
   ROUGE-2 F1:        0.00

REF:  'There are several moving vehicles on the road at the roundabout, There are several buildings and some trees and some lawns around the roundabout .'
HYP:  'In this image we can see a house, trees, vehicles on the road, fence, grass and also the road.'
   ROUGE-2 Recall:    12.00
   ROUGE-2 Precision: 13.04
   ROUGE-2 F1:        12.50

REF:  'The roundabout is on the 

In [ ]:
# Report overall averages
avg_r = sum(all_recalls) / len(all_recalls)
avg_p = sum(all_precisions) / len(all_precisions)
avg_f = sum(all_f1s) / len(all_f1s)
print("=== AVERAGE ROUGE-2 METRICS ===")
print(f"Recall:    {avg_r*100:.2f}")
print(f"Precision: {avg_p*100:.2f}")
print(f"F1:        {avg_f*100:.2f}")

=== AVERAGE ROUGE-2 METRICS ===
Recall:    10.81
Precision: 5.30
F1:        6.66


In [ ]:
# Compute ROUGE-3
all_recalls, all_precisions, all_f1s = [], [], []
for refs, pred in zip(all_references, predictions):
    recalls_per_sample, precisions_per_sample, f1s_per_sample = [], [], []
    for ref in refs:
        r, p, f = rouge_n(ref, pred, n=3)
        recalls_per_sample.append(r)
        precisions_per_sample.append(p)
        f1s_per_sample.append(f)

    max_score = max(f1s_per_sample)
    max_index = f1s_per_sample.index(max_score)
    all_recalls.append(recalls_per_sample[max_index])
    all_precisions.append(precisions_per_sample[max_index])
    all_f1s.append(f1s_per_sample[max_index])
    print(f"REF:  {refs[max_index]!r}")
    print(f"HYP:  {pred!r}")
    print(f"   ROUGE-3 Recall:    {recalls_per_sample[max_index]*100:.2f}")
    print(f"   ROUGE-3 Precision: {precisions_per_sample[max_index]*100:.2f}")
    print(f"   ROUGE-3 F1:        {f1s_per_sample[max_index]*100:.2f}\n")

Streaming output truncated to the last 5000 lines.
   ROUGE-3 F1:        5.88

REF:  'The roundabout connects three roads and a sculpture is in the middle of the roundabout .'
HYP:  'A roundabout with a large green circle in the middle. The roundabout has a white line painted on the road, a white bus on the road, and'
   ROUGE-3 Recall:    7.14
   ROUGE-3 Precision: 3.57
   ROUGE-3 F1:        4.76

REF:  'The roundabout with three exits and entrances is in the residential area .'
HYP:  ''
   ROUGE-3 Recall:    0.00
   ROUGE-3 Precision: 0.00
   ROUGE-3 F1:        0.00

REF:  'There are several moving vehicles on the road at the roundabout, There are several buildings and some trees and some lawns around the roundabout .'
HYP:  'In this image we can see a house, trees, vehicles on the road, fence, grass and also the road.'
   ROUGE-3 Recall:    8.33
   ROUGE-3 Precision: 9.09
   ROUGE-3 F1:        8.70

REF:  'The roundabout is on the grass next to trees and buildings .'
HYP:  'A rounda

In [ ]:
# Report overall averages
avg_r = sum(all_recalls) / len(all_recalls)
avg_p = sum(all_precisions) / len(all_precisions)
avg_f = sum(all_f1s) / len(all_f1s)
print("=== AVERAGE ROUGE-3 METRICS ===")
print(f"Recall:    {avg_r*100:.2f}")
print(f"Precision: {avg_p*100:.2f}")
print(f"F1:        {avg_f*100:.2f}")

=== AVERAGE ROUGE-3 METRICS ===
Recall:    3.17
Precision: 1.38
F1:        1.79


In [ ]:
# Compute ROUGE-4
all_recalls, all_precisions, all_f1s = [], [], []
for refs, pred in zip(all_references, predictions):
    recalls_per_sample, precisions_per_sample, f1s_per_sample = [], [], []
    for ref in refs:
        r, p, f = rouge_n(ref, pred, n=4)
        recalls_per_sample.append(r)
        precisions_per_sample.append(p)
        f1s_per_sample.append(f)

    max_score = max(f1s_per_sample)
    max_index = f1s_per_sample.index(max_score)
    all_recalls.append(recalls_per_sample[max_index])
    all_precisions.append(precisions_per_sample[max_index])
    all_f1s.append(f1s_per_sample[max_index])
    print(f"REF:  {refs[max_index]!r}")
    print(f"HYP:  {pred!r}")
    print(f"   ROUGE-4 Recall:    {recalls_per_sample[max_index]*100:.2f}")
    print(f"   ROUGE-4 Precision: {precisions_per_sample[max_index]*100:.2f}")
    print(f"   ROUGE-4 F1:        {f1s_per_sample[max_index]*100:.2f}\n")

Streaming output truncated to the last 5000 lines.
   ROUGE-4 F1:        0.00

REF:  'The roundabout connects three roads and a sculpture is in the middle of the roundabout .'
HYP:  'A roundabout with a large green circle in the middle. The roundabout has a white line painted on the road, a white bus on the road, and'
   ROUGE-4 Recall:    0.00
   ROUGE-4 Precision: 0.00
   ROUGE-4 F1:        0.00

REF:  'The roundabout with three exits and entrances is in the residential area .'
HYP:  ''
   ROUGE-4 Recall:    0.00
   ROUGE-4 Precision: 0.00
   ROUGE-4 F1:        0.00

REF:  'There are several moving vehicles on the road at the roundabout, There are several buildings and some trees and some lawns around the roundabout .'
HYP:  'In this image we can see a house, trees, vehicles on the road, fence, grass and also the road.'
   ROUGE-4 Recall:    4.35
   ROUGE-4 Precision: 4.76
   ROUGE-4 F1:        4.55

REF:  'The roundabout connects five roads and some houses are next to the roundabout

In [ ]:

# Report overall averages
avg_r = sum(all_recalls) / len(all_recalls)
avg_p = sum(all_precisions) / len(all_precisions)
avg_f = sum(all_f1s) / len(all_f1s)
print("=== AVERAGE ROUGE-4 METRICS ===")
print(f"Recall:    {avg_r*100:.2f}")
print(f"Precision: {avg_p*100:.2f}")
print(f"F1:        {avg_f*100:.2f}")

=== AVERAGE ROUGE-4 METRICS ===
Recall:    0.71
Precision: 0.31
F1:        0.39
